# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb langchain langchain_community langchain_openai

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [4]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [5]:
llm = OpenAI(temperature=0)

In [6]:
from pathlib import Path
import requests

url = "https://raw.githubusercontent.com/martin-gorner/tensorflow-rnn-shakespeare/master/shakespeare/sonnets.txt"
root_path = Path(".").absolute()
sonnets_path = root_path / "sonnets.txt"
response = requests.get(url)
with open("sonnets.txt", "w", encoding="utf-8") as f:
	f.write(response.text)

In [7]:
relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [8]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [9]:
sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [10]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [12]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [13]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [14]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about the most recent sonnet. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff Sonnets QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [15]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

/var/folders/s7/8db2fs716sj67tmf9cc2npbm0000gn/T/ipykernel_82273/1834837320.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [16]:
agent.invoke(
    "What's the third sonnet?"
)



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to get the third sonnet
Action: Sonnets QA System
Action Input: "What is the third sonnet?"
Observation:  The third sonnet is Sonnet XLVI.
Thought: I now know the final answer
Final Answer: Sonnet XLVI

> Finished chain.


{'input': "What's the third sonnet?", 'output': 'Sonnet XLVI'}

In [17]:
agent.invoke("Can you show me the fourth sonnet?")



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "Can you show me the fourth sonnet?"
Observation:  I don't know.
Thought: I should try again with a different question.
Action: Sonnets QA System
Action Input: "What is the fourth sonnet?"
Observation:  I don't know.
Thought: I should try again with a different question.
Action: Sonnets QA System
Action Input: "What is the fourth sonnet in the collection?"
Observation:  The fourth sonnet in the collection is Sonnet XLVI.
Thought: I now know the final answer.
Final Answer: Sonnet XLVI.

> Finished chain.


{'input': 'Can you show me the fourth sonnet?', 'output': 'Sonnet XLVI.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [18]:
tools = [
    Tool(
        name="Sonnet QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about the most recent sonnet. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff Sonnet QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [19]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [20]:
agent.invoke(
    "In which sonnet can we find 'Unthrifty loveliness'?"
)



> Entering new AgentExecutor chain...
 We should search for the phrase 'Unthrifty loveliness' in the text of each sonnet.
Action: Sonnet QA System
Action Input: "In which sonnet can we find 'Unthrifty loveliness'?"
Observation:  Sonnet II.


> Finished chain.


{'input': "In which sonnet can we find 'Unthrifty loveliness'?",
 'output': ' Sonnet II.'}

In [21]:
agent.invoke("Which sonnet is the best?")



> Entering new AgentExecutor chain...
 You should always think about what to do
Action: Sonnet QA System
Action Input: "Which sonnet is the best?"
Observation:  I don't know.


> Finished chain.


{'input': 'Which sonnet is the best?', 'output': " I don't know."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [22]:
tools = [
    Tool(
        name="Sonnet QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about the most recent sonnet. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff Sonnet QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [23]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [24]:
agent.invoke(
    "What sonnet can be read in Jupyter Notebook?"
)



> Entering new AgentExecutor chain...
 You should always think about what to do
Action: Sonnet QA System
Action Input: "What sonnet can be read in Jupyter Notebook?"
Observation:  I don't know.
Thought: Maybe I should try a different question.
Action: Sonnet QA System
Action Input: "What sonnet can be read in Jupyter Notebook?"
Observation:  I don't know.
Thought: Maybe I should try a different tool.
Action: Ruff Sonnet QA System
Action Input: "What sonnet can be read in Jupyter Notebook?"
Observation:  Ruff does not support Jupyter Notebooks.
Thought: Maybe I should try a different tool.
Action: Sonnet QA System
Action Input: "What sonnet can be read in Jupyter Notebook?"
Observation:  I don't know.
Thought: Maybe I should try a different question.
Action: Sonnet QA System
Action Input: "What sonnet can be read in Jupyter Notebook?"
Observation:  I don't know.
Thought: I now know the final answer.
Final Answer: There is no specific sonnet that can be read in Jupyter Notebook.

> Fin

{'input': 'What sonnet can be read in Jupyter Notebook?',
 'output': 'There is no specific sonnet that can be read in Jupyter Notebook.'}